# Two-Dimension Blob

This is an example usage of the skydy package.

This is an object that can move in two-dimensions. It has two input forces, co-linear with the
x- and y-coordinates.

In [1]:
import skydy
from skydy.connectors import DOF, Connection, Joint
from skydy.multibody import MultiBody
from skydy.rigidbody import (
    Body,
    BodyCoordinate,
    BodyForce,
    BodyTorque,
    Ground,
    GroundCoordinate,
)
import matplotlib.pyplot as plt
import pprint


## Define the car Body

Name and create the vehicle

In [2]:
# Give the car a name
car_name = "1"

# Define the body
b_car = Body(car_name)


Give the Body some dimensions.

In [3]:
l_car = 2
w_car = 2
h_car = 1

# Instantiate the car's dimensions
b_car.dims.assign_values([l_car, w_car, h_car])

In [4]:
# As it stands, this is simply a rigid body with 6 degrees of freedom and some geometry.
b_car.positions()

Matrix([
[    x^1_G(t)],
[    y^1_G(t)],
[    z^1_G(t)],
[theta^1_x(t)],
[theta^1_y(t)],
[theta^1_z(t)]])

In [5]:
# Check the dimensions of the body.
b_car.dims.as_dict()

{l^1_x: 2, l^1_y: 2, l^1_z: 1}

## Mass and Inertia Matrices

A Body comes equipped with a Mass and Inertia matrices. These are purely symbolic, but necessary to calculate the energy of the object. We do not physically calculate the moments of inertia, however, since we can define the Body in anyway, we align the body fixed axis with the principal inertial axes.

In [6]:
b_car.mass_matrix.as_mat()

Matrix([
[m_1,   0,   0],
[  0, m_1,   0],
[  0,   0, m_1]])

In [7]:
b_car.inertia_matrix.as_mat()

Matrix([
[I^1_xx, I^1_yx, I^1_zx],
[I^1_xy, I^1_yy, I^1_zy],
[I^1_xz, I^1_yz, I^1_zz]])

## External Force

Add a force at the Centre of Mass of the car.

In [8]:
# We can now add a force to the car.
# Add force to car in the car's x-coordinaate.
F1 = BodyForce(name="1", x_dir=True)

# Force is applied at the COM
F1_loc = BodyCoordinate("PF1", 0, 0, 0)

# Add force to car in the car's x-coordinaate.
F2 = BodyForce(name="2", y_dir=True)

# Force is applied at the COM
F2_loc = BodyCoordinate("PF1", 0, 0, 0)

# Add the force at the location
b_car.add_force(F1, F1_loc)
b_car.add_force(F2, F2_loc)

## Define the Ground Body and Coordinate.

This acts as the reference point for our body, i.e., allows us to propagate relative positions into absolute.

In [9]:
b_gnd = Ground()
p_gnd = GroundCoordinate()

In [10]:
# Define the location of the car to the ground, in the cars body frame.
p_car = BodyCoordinate("G1/O", 0, 0, 0)

## Degrees of Freedom

Define the coordinates that can move. We can also specify if a coordinate is fixed at a constant value. If the DOF is NOT specified, it is assumed to be fixed, with zero value.

We can move in the x- and y-coordinate of the base joint frame, i.e., the global coordinate system.
Note, index 0 represent the x-axis, 1 the y-, 2 the z-, 3 rotations about x-, 4 rotations about y- and 5 rotations about z-.

In [11]:
car_dofs = [DOF(0), DOF(1)]

## Joint

This is the object that connects the two bodies. Eaach coordinate is in its respective body frame.

In [12]:
j1 = Joint(p_gnd, p_car, car_dofs, name=p_gnd.name)

## Connection

In [13]:
cnx_car = Connection(b_gnd, j1, b_car)

## MultiBody

In [14]:
twod_car = MultiBody([cnx_car,])

## Coordinates

In [15]:
twod_car.coordinates

Matrix([
[x^1_G(t)],
[y^1_G(t)]])

## Energy

In [16]:
twod_car.kinetic_energy

0.5*m_1*(Derivative(x^1_G(t), t)**2 + Derivative(y^1_G(t), t)**2)

In [17]:
twod_car.potential_energy

0

## Kinetic Energy Metric

In [18]:
twod_car.G

Matrix([
[1.0*m_1,       0],
[      0, 1.0*m_1]])

## Equations of Motion

This returns the left and right hand sides of the equations of motion. The LHS will be linear in accelerations. The RHS is a function of the coordindates and inputs.

In [19]:
lhs, rhs = twod_car.eoms()
lhs

Matrix([
[1.0*m_1*Derivative(x^1_G(t), (t, 2))],
[1.0*m_1*Derivative(y^1_G(t), (t, 2))]])

In [20]:
rhs

Matrix([
[F^1_x],
[F^2_y]])

## Equilibria

Get the equilibria, either forced or unforced, for the described system.

In [21]:
twod_car.get_equilibria()

([], [])

## System Representation

Generate the equations of the following form:

- Nonlinear: $G \dot{x} = f(x, u)$,
- Linear: $G \dot{x} = A x + B u$,

where $G$ is the Kinetic Energy Metric. Note, x represents the positions and velocities, A is the state-transition matrix and B is the control input matrix.

### Nonlinear

In [22]:
# Nonlinear
A, B = twod_car.system_matrices(linearized=False)
A

Matrix([
[Derivative(x^1_G(t), t)],
[Derivative(y^1_G(t), t)],
[                  F^1_x],
[                  F^2_y]])

### Linear(ised)

In [25]:
# Linear
A, B = twod_car.system_matrices(linearized=True)
A

Matrix([
[0, 0, 1, 0],
[0, 0, 0, 1],
[0, 0, 0, 0],
[0, 0, 0, 0]])

In [26]:
B

Matrix([
[0, 0],
[0, 0],
[0, 1],
[1, 0]])

## Configuraton of bodies

In [27]:
pprint.pp(twod_car.get_configuration())

{'0': {'coords': {x^0_G(t): 0.0,
                  y^0_G(t): 0.0,
                  z^0_G(t): 0.0,
                  theta^0_x(t): 0.0,
                  theta^0_y(t): 0.0,
                  theta^0_z(t): 0.0},
       'dims': {l^0_x: 0, l^0_y: 0, l^0_z: 0}},
 '1': {'coords': {x^1_G(t): 1.0,
                  y^1_G(t): 1.0,
                  z^1_G(t): 0.0,
                  theta^1_x(t): 0.0,
                  theta^1_y(t): 0.0,
                  theta^1_z(t): 0.0},
       'dims': {l^1_x: 2, l^1_y: 2, l^1_z: 1}}}
